In [1]:
import pickle
import math
import time
import argparse
import random
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.functional import gumbel_softmax
from utils import gumbel_softmax_3d
%matplotlib inline

In [2]:
# 梯度优化算法类
class my_parameters(nn.Module):
    
    def __init__(self, batch_size, n):  
        super(my_parameters, self).__init__()
        self.batch_size = batch_size
        self.n = n
        if torch.cuda.is_available():
            device = 'cuda'
            rand = torch.randn(batch_size, n, 1, device=device) * 1e-5
        else:
            device = 'cpu'
            rand = torch.randn(batch_size, n, 1, requires_grad = True) * 1e-5        
        self.init_value = rand
        self.pps = Parameter(self.init_value)
        
    def expand_tensor(self, x):
        return x.unsqueeze(0).repeat(self.batch_size, 1)

In [3]:
def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    G = nx.from_dict_of_lists(data)
    return G

In [4]:
def trainGE(args, G):
    bs = args.batch_size
    n = G.number_of_nodes()
    A = nx.to_numpy_matrix(G)
    A = torch.from_numpy(A)
    A = A.type(torch.float32)
    best_log = []
    A = A.cuda()

    # set parameters
    #if torch.cuda.is_available():
        #A = A.cuda()
        #x = torch.randn(bs, n, 1, device='cuda')*1e-5
        #x.requires_grad = True
    nnn = my_parameters(bs, n)

    # set optimizer
    optimizer = torch.optim.Adam(nnn.parameters(), lr=args.lr)

    # training
    cost_arr = []
    log_min = [0]
    log_mean = [0]
    bc = torch.zeros(bs)
    
    for _ in range(args.iterations):
        optimizer.zero_grad()
        if torch.cuda.is_available():
            probs = torch.empty(bs, n, 2, device='cuda')
        else:
            probs = torch.empty(bs, n, 2)
        p = torch.sigmoid(nnn.pps)
        probs[:, :, 0] = p.squeeze()
        probs[:, :, -1] = 1-probs[:, :, 0]
        logits = torch.log(probs+1e-10)
        s = gumbel_softmax_3d(logits, tau=args.tau, hard=args.hard)[:, :, 0]
        s = torch.unsqueeze(s, -1)  # size [bs, n, 1]
        cost = -1 * torch.sum(s)
        constraint = torch.sum(torch.transpose(s, 1, 2) @ A @ s)
        loss = cost + args.eta * constraint
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            cost_ = -1 * torch.sum(s, dim=1).squeeze()
            constraint_ = torch.squeeze(torch.transpose(s, 1, 2) @ A @ s)
            constraint = constraint_.cpu().numpy()
            loss_ = cost_ + constraint_
            #print(loss_)
            #if _ % 100 == 0:
                #print(loss_)
                #print(max(loss_) - min(loss_))
            
            idx = np.argwhere(constraint == 0)  # select constraint=0
            
            if _ > 9500 and _ % 100 ==0:
                print('cost:',torch.sort(cost_))
                print('constraint:',torch.sort(constraint_))
                print('loss:',torch.sort(loss_))
            

                
            #交叉变异
            if _ % 10000 == 0 and _ >= 10000:
                
                ratio = 1/4
                r = int(ratio * bs)
                m = 2e-3
                
                maxdx = torch.argsort(loss_, dim=0,descending=True).squeeze()
                mindx = torch.argsort(loss_, dim=0,descending=False).squeeze()

                #print('maxdx:',maxdx)
                #print('mindx:',mindx)
                L1 = random.sample(list(maxdx[0:r]), r)
                L2 = random.sample(list(mindx[0:r]), r)
                #print(L1)
                #print(L2)
                #print(loss_[L1[0]])
                for i in range(r//2):
                    #print(L1[i])
                    #print('before:',nnn.pps.data[L1[i], : ,0])
                    #print(L2[i])
                    #print('father:',nnn.pps.data[L2[i], :, 0])
                    #print(L2[-(i+1)])
                    #print('mother:',nnn.pps.data[L2[-(i+1)], :, 0])
                    rand = torch.rand(n).cuda()
                    nnn.pps.data[L1[i], : ,0] = torch.where(rand < 0.5, 
                                                            nnn.pps.data[L2[i], :, 0], 
                                                            nnn.pps.data[L2[r-1-i], :, 0])
                    nnn.pps.data[L1[r-1-i], :, 0] = torch.where(rand < 0.5, 
                                                            nnn.pps.data[L2[r-1-i], :, 0], 
                                                            nnn.pps.data[L2[i], :, 0])
                    #print('after:',nnn.pps.data[L1[i], : ,0])
                    rand = torch.rand(n).cuda()
                    nnn.pps.data[L1[i], : ,0] = torch.where(rand < m, 
                                                            nnn.pps.data[L1[i], : ,0], 
                                                            torch.randn_like(nnn.pps.data[L1[i], : ,0]) * 1e-5)
                    #count = torch.where(rand<m,torch.ones_like(rand),torch.zeros_like(rand))
                    #print('mutation_count:',torch.sum(count))



            if len(idx) != 0:
                s = gumbel_softmax_3d(logits, tau=args.tau, hard=True)[:, :, 0]
                s = torch.unsqueeze(s, -1).cpu()  # size [bs, n, 1]
                cost_ = -1 * torch.sum(s, dim=1)
                cost = cost_[idx.reshape(-1,)]
                # from size [bs, 1] select constrain=0
                cost_arr.append(torch.min(cost.cpu()).item())
                log_min.append(torch.min(cost.cpu()).item())
                log_mean.append(torch.mean(cost.cpu()).item())
                


                                    
            else:
                log_min.append(log_min[-1])
                log_mean.append(log_mean[-1])
                
                              
                        
            if _ % 100 == 0:
                print(_)
                if len(cost_arr) != 0:
                    print('# {}, cost: {}'.format(_, ((np.sort(cost_arr))[0:8])))
                else:
                    print('Failed!')
                    

    return cost_arr,log_min,log_mean

In [5]:
def main():
    # Training settings
    parser = argparse.ArgumentParser(
        description='Solving MIS problems (with fixed tau in GS, parallel version)')
    parser.add_argument('--batch-size', type=int, default=128,
                        help='batch size (default: 128)')
    parser.add_argument('--data', type=str, default='citeseer',
                        help='data name (default: cora)')
    parser.add_argument('--tau', type=float, default=1.,
                        help='tau value in Gumbel-softmax (default: 1)')
    parser.add_argument('--hard', type=bool, default=True,
                        help='hard sampling in Gumbel-softmax (default: True)')
    parser.add_argument('--lr', type=float, default=1e-2,
                        help='learning rate (default: 1e-2)')
    parser.add_argument('--eta', type=float, default=3.,
                        help='constraint (default: 5)')
    parser.add_argument('--ensemble', type=int, default=5,
                        help='# experiments (default: 100)')
    parser.add_argument('--iterations', type=int, default=30000,
                        help='# iterations in gradient descent (default: 20000)')
    parser.add_argument('--seed', type=int, default=1, help='random seed (default: 1)')
    args = parser.parse_args(args=[])

    # torch.manual_seed(args.seed)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print(device)

    # loading data
    G = load_data('./data/ind.' + args.data + '.graph')

    for i in range(args.ensemble):
        (cost1,logmin1,logmean1) = trainGE(args, G)
        del logmin1[0]
        del logmean1[0]
        if len(cost1) != 0:
            print('# {}, min_cost: {}'.format(i, min(cost1)))
        else:
            print('Failed!')
            
        plt.figure(figsize=(16,16))
        plt.plot(logmin1, label='mindx')
        plt.xlim(0, 20000)
        plt.ylim(-1900, 100)
        plt.legend()
        plt.show()
        plt.figure(figsize=(16,16))
        plt.plot(logmean1, label='meandx')
        plt.xlim(0, 20000)
        plt.ylim(-1900, 100)
        plt.legend()
        plt.show()
        plt.figure(figsize=(16,16))
        plt.plot(cost1, label='cost')
        plt.xlim(0, 20000)
        plt.ylim(-1900, 100)
        plt.legend()
        plt.show()
        

if __name__ == '__main__':
    main()

cuda


KeyboardInterrupt: 